In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import cv2
import time
import os
import glob

In [ ]:
test_dir = '../data/test'
save_dir = '../model/alexnet'
size = (227, 227)

In [ ]:
name_id_map = {'cat': 0, 'dog': 1}
id_name_map = {id:name for name, id in name_id_map.items()}
n_classes = len(name_id_map)
logging.info('name_id_map: %s', name_id_map)
logging.info('id_name_map: %s', id_name_map)
logging.info('n_classes: %d', n_classes)

### input layer

In [ ]:
# Input: N*227*227*3
channels0 = 3
x_ = tf.placeholder(tf.float32, [None, 227, 227, channels0])
y_ = tf.placeholder(tf.int32, [None, n_classes])
logging.info('x_.shape = %s', x_.shape.as_list())
logging.info('y_.shape = %s', y_.shape.as_list())

### 1st layer: conv1(bn) => lrn1 => pool1

In [ ]:
with tf.variable_scope('layer1'):
    # C1: N*227*227*3 => C11*11s4*4V => N*55*55*96
    channels1 = 96
    weight1 = tf.Variable(tf.truncated_normal([11, 11, channels0, channels1], stddev=0.01), name='weight1')
    conv1 = tf.nn.conv2d(x_, filter=weight1, strides=[1, 4, 4, 1], padding='VALID')
    # compare with bias
    # bias1 = tf.Variable(tf.constant(0.1, dtype=tf.float32, shape=[channels1]))
    # conv1 = tf.nn.bias_add(conv1, bias1)
    # batch normalization
    # y = (x - batch_mean) / sqrt(batch_var + epsilon)
    # z = scale * y + offset
    mean1 = tf.Variable(tf.zeros([channels1]), trainable=False, name='mean1')
    var1 = tf.Variable(tf.ones([channels1]), trainable=False, name='var1')
    epsilon1 = 0.0001
    scale1 = tf.Variable(tf.ones([channels1]), name='scale1')
    offset1 = tf.Variable(tf.zeros(channels1), name='offset1')
    conv1 = tf.nn.batch_normalization(conv1, mean1, var1, offset1, scale1, epsilon1)
    conv1 = tf.nn.relu(conv1)
    logging.info('weight1: name=%s, shape=%s', weight1.name, weight1.shape.as_list())
    logging.info('mean1: name=%s, shape=%s', mean1.name, mean1.shape.as_list())
    logging.info('var1: name=%s, shape=%s', var1.name, var1.shape.as_list())
    logging.info('scale1: name=%s, shape=%s', scale1.name, scale1.shape.as_list())
    logging.info('offset1: name=%s, shape=%s', offset1.name, offset1.shape.as_list())
    logging.info('conv1.shape: %s', conv1.shape.as_list())

    # L1: N*55*55*96 => LRN => N*55*55*96 
    # sqr_sum[a, b, c, d] = sum(input[a, b, c, d-depth_radius:d+depth_radius+1] ** 2)
    # output = input / ((bias + alpha * sqr_sum) ** beta)
    lrn1 = tf.nn.lrn(conv1, depth_radius=4, bias=1.0, alpha=0.001/9, beta=0.75)
    logging.info('lrn1.shape: %s', lrn1.shape.as_list())

    # P1: N*55*55*96 => P3*3s2*2V => N*27*27*96
    pool1 = tf.nn.max_pool(lrn1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')
    logging.info('pool1.shape: %s', pool1.shape.as_list())


### 2nd layer: conv2（bn） => lrn2 => pool2

In [ ]:
with tf.variable_scope('layer2'):
    # C2: N*27*27*96 => C5*5s1*1S => N*27*27*256
    channels2 = 256
    weight2 = tf.Variable(tf.truncated_normal([5, 5, channels1, channels2], stddev=0.01), name='weight2')
    conv2 = tf.nn.conv2d(pool1, filter=weight2, strides=[1, 1, 1, 1], padding='SAME')
    # compare with bias
    # bias2 = tf.Variable(tf.constant(0.1, dtype=tf.float32, shape=[channels2]))
    # conv2 = tf.nn.bias_add(conv2, bias2)
    # batch normalization
    # y = (x - batch_mean) / sqrt(batch_var + epsilon)
    # z = scale * y + offset
    mean2 = tf.Variable(tf.zeros([channels2]), trainable=False, name='mean2')
    var2 = tf.Variable(tf.ones([channels2]), trainable=False, name='var2')
    epsilon2 = 0.0001
    scale2 = tf.Variable(tf.ones([channels2]), name='scale2')
    offset2 = tf.Variable(tf.zeros(channels2), name='offset2')
    conv2 = tf.nn.batch_normalization(conv2, mean2, var2, offset2, scale2, epsilon2)
    conv2 = tf.nn.relu(conv2)
    logging.info('weight2: name=%s, shape=%s', weight2.name, weight2.shape.as_list())
    logging.info('mean2: name=%s, shape=%s', mean2.name, mean2.shape.as_list())
    logging.info('var2: name=%s, shape=%s', var2.name, var2.shape.as_list())
    logging.info('scale2: name=%s, shape=%s', scale2.name, scale2.shape.as_list())
    logging.info('offset2: name=%s, shape=%s', offset2.name, offset2.shape.as_list())
    logging.info('conv2.shape: %s', conv2.shape.as_list())

    # L2: N*27*27*256 => LRN => N*27*27*256 
    # sqr_sum[a, b, c, d] = sum(input[a, b, c, d-depth_radius:d + depth_radius+1] ** 2)
    # output = input / ((bias + alpha * sqr_sum) ** beta)
    lrn2 = tf.nn.lrn(conv2, depth_radius=4, bias=1.0, alpha=0.001/9, beta=0.75)
    logging.info('lrn2.shape: %s', lrn2.shape.as_list())

    # P2: N*27*27*256 => P3*3s2*2V => N*13*13*256
    pool2 = tf.nn.max_pool(lrn2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')
    logging.info('pool2.shape: %s', pool2.shape.as_list())

### 3rd layer: conv3(bn)

In [ ]:
with tf.variable_scope('layer3'):
    # C3: N*13*13*256 => C3*3s1*1S => N*13*13*384
    channels3 = 384
    weight3 = tf.Variable(tf.truncated_normal([3, 3, channels2, channels3], stddev=0.01), name='weight3')
    conv3 = tf.nn.conv2d(pool2, filter=weight3, strides=[1, 1, 1, 1], padding='SAME')
    # compare with bias
    # bias3 = tf.Variable(tf.constant(0.1, dtype=tf.float32, shape=[channels3]))
    # conv3 = tf.nn.bias_add(conv3, bias3)
    # batch normalization
    # y = (x - batch_mean) / sqrt(batch_var + epsilon)
    # z = scale * y + offset
    mean3 = tf.Variable(tf.zeros([channels3]), trainable=False, name='mean3')
    var3 = tf.Variable(tf.ones([channels3]), trainable=False, name='var3')
    epsilon3 = 0.0001
    scale3 = tf.Variable(tf.ones([channels3]), name='scale3')
    offset3 = tf.Variable(tf.zeros(channels3), name='offset3')
    conv3 = tf.nn.batch_normalization(conv3, mean3, var3, offset3, scale3, epsilon3)
    conv3 = tf.nn.relu(conv3)
    logging.info('weight3: name=%s, shape=%s', weight3.name, weight3.shape.as_list())
    logging.info('mean3: name=%s, shape=%s', mean3.name, mean3.shape.as_list())
    logging.info('var3: name=%s, shape=%s', var3.name, var3.shape.as_list())
    logging.info('scale3: name=%s, shape=%s', scale3.name, scale3.shape.as_list())
    logging.info('offset3: name=%s, shape=%s', offset3.name, offset3.shape.as_list())
    logging.info('conv3.shape: %s', conv3.shape.as_list())

### 4th layer: conv4(bn)

In [ ]:
with tf.variable_scope('layer4'):
    # C4: N*13*13*384 => C3*3s1*1S => N*13*13*384
    channels4 = 384
    weight4 = tf.Variable(tf.truncated_normal([3, 3, channels3, channels4], stddev=0.01), name='weight4')
    conv4 = tf.nn.conv2d(conv3, filter=weight4, strides=[1, 1, 1, 1], padding='SAME')
    # compare with bias
    # bias4 = tf.Variable(tf.constant(0.1, dtype=tf.float32, shape=[channels4]))
    # conv4 = tf.nn.bias_add(conv4, bias4)
    # batch normalization
    # y = (x - batch_mean) / sqrt(batch_var + epsilon)
    # z = scale * y + offset
    mean4 = tf.Variable(tf.zeros([channels4]), trainable=False, name='mean4')
    var4 = tf.Variable(tf.ones([channels4]), trainable=False, name='var4')
    epsilon4 = 0.0001
    scale4 = tf.Variable(tf.ones([channels4]), name='scale4')
    offset4 = tf.Variable(tf.zeros(channels4), name='offset4')
    conv4 = tf.nn.batch_normalization(conv4, mean4, var4, offset4, scale4, epsilon4)
    conv4 = tf.nn.relu(conv4)
    logging.info('weight4: name=%s, shape=%s', weight4.name, weight4.shape.as_list())
    logging.info('mean4: name=%s, shape=%s', mean4.name, mean4.shape.as_list())
    logging.info('var4: name=%s, shape=%s', var4.name, var4.shape.as_list())
    logging.info('scale4: name=%s, shape=%s', scale4.name, scale4.shape.as_list())
    logging.info('offset4: name=%s, shape=%s', offset4.name, offset4.shape.as_list())
    logging.info('conv4.shape: %s', conv4.shape.as_list())

### 5th layer: conv5(bn) => pool5 => reshape5

In [ ]:
with tf.variable_scope('layer5'):
    # C5: N*13*13*384 => C3*3s1*1S => N*13*13*256
    channels5 = 256
    weight5 = tf.Variable(tf.truncated_normal([3, 3, channels4, channels5], stddev=0.01), name='weight5')
    conv5 = tf.nn.conv2d(conv4, filter=weight5, strides=[1, 1, 1, 1], padding='SAME')
    # compare with bias
    # bias5 = tf.Variable(tf.constant(0.1, dtype=tf.float32, shape=[channels5]))
    # conv5 = tf.nn.bias_add(conv5, bias5)
    # batch normalization
    # y = (x - batch_mean) / sqrt(batch_var + epsilon)
    # z = scale * y + offset
    mean5 = tf.Variable(tf.zeros([channels5]), trainable=False, name='mean5')
    var5 = tf.Variable(tf.ones([channels5]), trainable=False, name='var5')
    epsilon5 = 0.0001
    scale5 = tf.Variable(tf.ones([channels5]), name='scale5')
    offset5 = tf.Variable(tf.zeros(channels5), name='offset5')
    conv5 = tf.nn.batch_normalization(conv5, mean5, var5, offset5, scale5, epsilon5)
    conv5 = tf.nn.relu(conv5)
    logging.info('weight5: name=%s, shape=%s', weight5.name, weight5.shape.as_list())
    logging.info('mean5: name=%s, shape=%s', mean5.name, mean5.shape.as_list())
    logging.info('var5: name=%s, shape=%s', var5.name, var5.shape.as_list())
    logging.info('scale5: name=%s, shape=%s', scale5.name, scale5.shape.as_list())
    logging.info('offset5: name=%s, shape=%s', offset5.name, offset5.shape.as_list())
    logging.info('conv5.shape: %s', conv5.shape.as_list())

    # P5: N*13*13*256 => P3*3s2*2V => N*6*6*256
    pool5 = tf.nn.max_pool(conv5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')
    logging.info('pool5.shape: %s', pool5.shape.as_list())

    # R5: N*6*6*256 => Reshape => N*9216
    nodes5 = 6 * 6 * 256
    reshape5 = tf.reshape(pool5, [-1, nodes5])
    logging.info('reshape5.shape: %s', reshape5.shape.as_list())

### 6th layer: full6(bn) => drop6

In [ ]:
with tf.variable_scope('layer6'):
    # F6: N*9216 => N*4096
    nodes6 = 4096
    weight6 = tf.Variable(tf.truncated_normal([nodes5, nodes6], stddev=0.1), name='weight6')
    full6 = tf.matmul(reshape5, weight6)
    # compare with bias
    # bias6 = tf.Variable(tf.constant(0.1, dtype=tf.float32, shape=[nodes6]))
    # full6 = tf.nn.bias_add(full6, bias6)
    # batch normalization
    # y = (x - batch_mean) / sqrt(batch_var + epsilon)
    # z = scale * y + offset
    mean6 = tf.Variable(tf.zeros([nodes6]), trainable=False, name='mean6')
    var6 = tf.Variable(tf.ones([nodes6]), trainable=False, name='var6')
    epsilon6 = 0.0001
    scale6 = tf.Variable(tf.ones([nodes6]), name='scale6')
    offset6 = tf.Variable(tf.zeros(nodes6), name='offset6')
    full6 = tf.nn.batch_normalization(full6, mean6, var6, offset6, scale6, epsilon6)
    full6 = tf.nn.relu(full6)
    logging.info('weight6: name=%s, shape=%s', weight6.name, weight6.shape.as_list())
    logging.info('mean6: name=%s, shape=%s', mean6.name, mean6.shape.as_list())
    logging.info('var6: name=%s, shape=%s', var6.name, var6.shape.as_list())
    logging.info('scale6: name=%s, shape=%s', scale6.name, scale6.shape.as_list())
    logging.info('offset6: name=%s, shape=%s', offset6.name, offset6.shape.as_list())
    logging.info('full6.shape: %s', full6.shape.as_list())

    # D6: N*4096 => N*4096
    # the dropped elements become 0, and the kept elements scaled 
    # up by 1/keep_prob, which will unchange the expected sum.
    keep_prob6 = 0.5
    drop6 = tf.nn.dropout(full6, keep_prob6)
    logging.info('drop6.shape: %s', drop6.shape.as_list())

### 7th layer: full7(bn) => drop7

In [ ]:
with tf.variable_scope('layer7'):
    # F7: N*4096 => N*2048
    nodes7 = 2048
    weight7 = tf.Variable(tf.truncated_normal([nodes6, nodes7], stddev=0.1), name='weight7')
    full7 = tf.matmul(drop6, weight7)
    # compare with bias
    # bias7 = tf.Variable(tf.constant(0.1, dtype=tf.float32, shape=[nodes7]))
    # full7 = tf.nn.bias_add(full7, bias7)
    # batch normalization
    # y = (x - batch_mean) / sqrt(batch_var + epsilon)
    # z = scale * y + offset
    mean7 = tf.Variable(tf.zeros([nodes7]), trainable=False, name='mean7')
    var7 = tf.Variable(tf.ones([nodes7]), trainable=False, name='var7')
    epsilon7 = 0.0001
    scale7 = tf.Variable(tf.ones([nodes7]), name='scale7')
    offset7 = tf.Variable(tf.zeros(nodes7), name='offset7')
    full7 = tf.nn.batch_normalization(full7, mean7, var7, offset7, scale7, epsilon7)
    full7 = tf.nn.relu(full7)
    logging.info('weight7: name=%s, shape=%s', weight7.name, weight7.shape.as_list())
    logging.info('mean7: name=%s, shape=%s', mean7.name, mean7.shape.as_list())
    logging.info('var7: name=%s, shape=%s', var7.name, var7.shape.as_list())
    logging.info('scale7: name=%s, shape=%s', scale7.name, scale7.shape.as_list())
    logging.info('offset7: name=%s, shape=%s', offset7.name, offset7.shape.as_list())
    logging.info('full7.shape: %s', full7.shape.as_list())

    # D7: N*2048 => N*2048
    # the dropped elements become 0, and the kept elements scaled 
    # up by 1/keep_prob, which will unchange the expected sum.
    keep_prob7 = 0.5
    drop7 = tf.nn.dropout(full7, keep_prob7)
    logging.info('drop7.shape: %s', drop7.shape.as_list())

### 8th layer: full8(bn)

In [ ]:
with tf.variable_scope('layer8'):
    # F8: N*2048 => N*2
    nodes8 = n_classes
    weight8 = tf.Variable(tf.truncated_normal([nodes7, nodes8], stddev=0.1), name='weight8')
    full8 = tf.matmul(drop7, weight8)
    # compare with bias
    bias8 = tf.Variable(tf.constant(0.1, dtype=tf.float32, shape=[nodes8]), name='bias8')
    full8 = tf.nn.bias_add(full8, bias8)
    logging.info('weight8: name=%s, shape=%s', bias8.name, bias8.shape.as_list())
    logging.info('bias8: name=%s, shape=%s', bias8.name, bias8.shape.as_list())
    logging.info('full8.shape: %s', full8.shape.as_list())

### output layer: softmax

In [ ]:
y = tf.nn.softmax(full8)
logging.info('y.shape: %s', y.shape.as_list())

### predict

In [ ]:
def predict(image):
    image_one_list = image.reshape(-1, *image.shape)
    with tf.Session() as sess:
        #sess.run(tf.global_variables_initializer())
        model = tf.train.latest_checkpoint(save_dir)
        saver = tf.train.Saver()
        saver.restore(sess, model)
        y_predict = sess.run(y, {x_:image_one_list})
        id_predict = np.argmax(y_predict)
        name_predict = id_name_map[id_predict]
        logging.info('y_predict=%s, id_predict=%d, name_predict=%s', y_predict, id_predict, name_predict)
        plt.figure()
        plt.imshow(image)
        plt.title("it's a %s"%(name_predict))
    return name_predict

In [ ]:
for image_path in glob.glob(os.path.join(test_dir, '*.jpg')):
    image = cv2.imread(image_path)
    image = cv2.resize(image, size)
    label_predict = predict(image)
    logging.info('image: %s, label_predict: %s', image_path, label_predict)